## ExPECA Testbed HI Setup

---

**Overview**

Read README.md first to understand the purpose of this notebook and the setup.  
This notebook will go through the steps for creating the HI setup needed for the ExPECA testbed.

**The notebook will create a setup with the following components**

- Edge Device: Start a containerized application that runs the edge device in the ExPECA testbed.
- Edge Server: Start a containerized application that runs the edge server in the ExPECA testbed.
- Connection: SSH connection to the Edge Device and Edge Server.
- Start the external controller for the experiment.

**Steps to follow:**

1. Setup authentication for the ExPECA testbed.
2. Install required packages.
3. Import the required libraries.
4. Reserve the ExPECA testbed equipment for Edge Server
5. Load and run the Edge Server container.
6. Reserve the ExPECA testbed equipment for Edge Device
7. Load and run the Edge Device container.
    - With Edge Server public IP environment variable set.
8. Start the external controller for the experiment.


### 1. Setup Authentication for the ExPECA Testbed

---

1. Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/).
2. Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [ ]:
import os, re
from getpass import getpass

# Replace with your OpenStack credentials file path
with open('henrik-project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

### 2. Install Required Packages

---

Install required packages. Ignore the warnings.

In [ ]:
!pip uninstall -q -y moviepy
!pip install -q --upgrade pip setuptools wheel
!pip install -q jedi loguru
!pip install -q --no-deps git+https://github.com/ChameleonCloud/python-blazarclient.git@chameleoncloud/2023.1
!pip install -q fabric paramiko gnocchiclient \
    python-cinderclient python-glanceclient python-ironicclient \
    python-manilaclient python-neutronclient python-novaclient python-zunclient
!pip install -q --no-deps git+https://github.com/KTH-EXPECA/python-chi.git

### 3. Import the Required Libraries

---

Import the required libraries. Ignore the warnings.

In [6]:
import json
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, get_available_publicips, get_segment_ids, get_worker_interfaces

### 4. Reserve Worker for Edge Server

---

Next we reserve the ExPECA testbed equipment for the Edge Server.

In [ ]:
# Worker reservation for Edge Server
worker_name = "worker-09"

found = False
leaseslist = list_reservations(brief=True)
for lease_dict in leaseslist:
  if lease_dict["name"] == worker_name+"-lease":
    worker_reservation_id = lease_dict["reservation_id"]
    found = True

if not found:
  worker_lease = reserve(
    { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
  )
  worker_reservation_id = worker_lease["reservations"][0]["id"]

print(json.dumps(leaseslist,indent=4))

### 5. Load and Run the Edge Server Container

---

1. Reserve Public IP for the Edge Server.
2. Upload the Edge Server container image to the ExPECA testbed from Docker Hub.
3. Run the container.

In [ ]:
# check public IPs and select one
available_pub_ips = get_available_publicips()
pub_ip = available_pub_ips[0]
logger.info(f"Available public ips: {available_pub_ips}.")
logger.info(f"We choose {pub_ip} for this container.")
pub_ip = available_pub_ips[0]

# check available interfaces of the worker
interfaces = list(get_worker_interfaces(worker_name).values())[0]
available_ifs = []
for interface in interfaces.keys():
  if len(interfaces[interface]['connections']) == 0:
    available_ifs.append(interface)
logger.info(f"Available interfaces on {worker_name}: {available_ifs}")

# run the container
publicnet = chi.network.get_network("serverpublic")
container_name = "hi-edge-server"
chi.container.create_container(
    name = container_name,
    image = "h3nkk44/hi-framework-edge-server:latest_amd64",
    reservation_id = worker_reservation_id,
    environment = {
        "DNS_IP":"8.8.8.8",
        "GATEWAY_IP":"130.237.11.97",
        "PASS":"expeca"
    },
    mounts = [],
    nets = [
        { "network" : publicnet['id'] },
    ],
    labels = {
        "networks.1.interface":available_ifs[0],
        "networks.1.ip":pub_ip+"/27",
        "networks.1.gateway":"130.237.11.97",
        "capabilities.privileged":"true",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container, reachable at {pub_ip}.")

edge_server_pub_ip = pub_ip
logger.success(f"Edge Server public IP: {edge_server_pub_ip}")

### 6. Reserve Worker for Edge Device

---

Next we reserve the ExPECA testbed equipment for the Edge Server.

In [ ]:
# Worker reservation for Edge Device
worker_name = "worker-21"

found = False
leaseslist = list_reservations(brief=True)
for lease_dict in leaseslist:
  if lease_dict["name"] == worker_name+"-lease":
    worker_reservation_id = lease_dict["reservation_id"]
    found = True

if not found:
  worker_lease = reserve(
    { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
  )
  worker_reservation_id = worker_lease["reservations"][0]["id"]

print(json.dumps(leaseslist,indent=4))

### 7. Load and Run the Edge Device Container

---

1. Reserve Public IP for the Edge Device.
2. Upload the Edge Server container image to the ExPECA testbed from Docker Hub.
3. Run the container.
    - With Edge Server public IP environment variable set.

In [ ]:
# check public IPs and select one
available_pub_ips = get_available_publicips()
pub_ip = available_pub_ips[0]
logger.info(f"Available public ips: {available_pub_ips}.")
logger.info(f"We choose {pub_ip} for this container.")
pub_ip = available_pub_ips[0]

# check available interfaces of the worker
interfaces = list(get_worker_interfaces(worker_name).values())[0]
available_ifs = []
for interface in interfaces.keys():
  if len(interfaces[interface]['connections']) == 0:
    available_ifs.append(interface)
logger.info(f"Available interfaces on {worker_name}: {available_ifs}")

# run the container
publicnet = chi.network.get_network("serverpublic")
container_name = "hi-edge-device"
chi.container.create_container(
    name = container_name,
    image = "h3nkk44/hi-framework-edge-device:latest_arm64",
    reservation_id = worker_reservation_id,
    environment = {
        "DNS_IP":"8.8.8.8",
        "GATEWAY_IP":"130.237.11.97",
        "PASS":"expeca",
        "EDGE_SERVER_IP": edge_server_pub_ip
    },
    mounts = [],
    nets = [
        { "network" : publicnet['id'] },
    ],
    labels = {
        "networks.1.interface":available_ifs[0],
        "networks.1.ip":pub_ip+"/27",
        "networks.1.gateway":"130.237.11.97",
        "capabilities.privileged":"true",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container, reachable at {pub_ip}.")

#### 8. Start the External Controller for the Experiment

Now we can start the external controller for the experiment.
The controllers purpose is:
- Send experiment configuration to the Edge Device and Edge Server.
- Generate samples and send them to the Edge Device.
- Collect the results and logs from the Edge Device and Edge Server.

---

#### Connecting to the Edge Device and Edge Server using SSH

*NOTE: You can already use the public IP address of the Edge Device and Edge Server to connect to them.
You don't need to use SSH to run the External Controllers.*

- Using the Chameleon interface it is easy to monitor and control the containers.

Now you can ssh into the containers with the following command:
```
ssh root@[public_ip_address]
```
From anywhere. Also, you can run a remote command here.  
The result will be printed on the logs page in Chameleon.

---
